<a href="https://colab.research.google.com/github/mricos/bird2vec/blob/master/bird2vec_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==1.12.0l

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0\
/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo dpkg -i /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
print('add key: ok')
!sudo apt-get update
!sudo apt-get install cuda-libraries-9-0

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Load the model
import tensorflow as tf

In [0]:
!cat /content/gdrive/My\ Drive/wgtrain-stream-kauai-6-2sec/checkpoint

In [0]:
path="/content/gdrive/My Drive/wgtrain-stream-kauai-6-2sec/"
tf.reset_default_graph()
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph(path+'infer/infer.meta')
graph = tf.get_default_graph()
saver.restore(sess, path+'model.ckpt-2403')

In [0]:
# Generate and display audio

# CHANGE THESE to change number of examples generated/displayed
ngenerate = 64
ndisplay = 4

import numpy as np
import PIL.Image
from IPython.display import display, Audio
import time as time

# Sample latent vectors (aka 64 jewel vectors w/ 100 values -1 to 1)
_z = (np.random.rand(ngenerate, 100) * 2.) - 1.  

In [0]:
# Generate
z = graph.get_tensor_by_name('z:0')
G_z = graph.get_tensor_by_name('G_z:0')[:, :, 0]
G_z_spec = graph.get_tensor_by_name('G_z:0')

In [0]:
start = time.time()
_G_z, _G_z_spec = sess.run([G_z, G_z_spec], {z:_z})
print('Finished! (Took {} seconds)'.format(time.time() - start))

In [0]:
#jewels=np.reshape(_z[61],(-1,100))
jewel_vectors= np.zeros([16,100]) # 16 rows, 100 columns of jewels
jewels[1,0] = 1.0  # first jewel vector. All zeros but first jewel is 1.0
jewels[2,50] = 1.0 # second jewel vector, 50th jewel is 1.0 all others zero
jewels[3,99]=.99   # third jewel vector, last jewel is 1.0
jewels[4,:]=.5     # forth jewel vector, all 0.5
jewels[5,:]=-.5     # fifth jewel vector, all -0.5
jewels[6] = (np.random.rand(1, 100) * .5) - .25  
jewels[7] = (np.random.rand(1, 100) * .25) - .125  


tubes, tubes_spec = sess.run([G_z, G_z_spec], {z:jewels })

In [0]:
plt.plot(tubes[0][2])

In [0]:
display(Audio(tubes[0][0],rate=16000)) # all zeros
display(Audio(tubes[0][1],rate=16000)) # all zeros but first jewel is 1.0
display(Audio(tubes[0][2],rate=16000)) # all zeros but 50th jewel is 1.0
display(Audio(tubes[0][3],rate=16000)) # all zero, last jewel is 1.0
display(Audio(tubes[0][4],rate=16000)) # forth jewel vector, all 0.5
display(Audio(tubes[0][5],rate=16000)) # fifth jewel vector, all -0.5
display(Audio(tubes[0][6],rate=16000)) # sixth jewel vector, random [-.5 .5]
display(Audio(tubes[0][7],rate=16000)) # sixth jewel vector, random [-.25 .25]

In [0]:
print("z: Type: {} shape: {}".format(type(z),z.shape))
print("G_z: Type: {} shape: {}".format(type(G_z),G_z.shape))
print("G_z_spec: Type: {} shape: {}".format(type(G_z_spec),G_z_spec.shape))
print("_z: Type: {} shape: {}".format(type(z),_z.shape))
print("jewels: Type: {} shape: {}".format(type(z),jewels.shape))
print("_G_z: Type: {} shape: {}".format(type(G_z),_G_z.shape))
print("_G_z_spec: Type: {} shape: {}".format(type(_G_z_spec),_G_z_spec.shape))
print("All 100 elements of first latent vector:\n{}".format(_z[0,:]))
print("All 100 elements of second latent vector:\n{}".format(_z[1,0:len(_z[0])]))

In [0]:
for i in range(0,2):
  print('-' * 80)
  print('Example {}'.format(i))
  display(Audio(_G_z[i], rate=16000))

In [0]:
import librosa

!mkdir -p "/content/gdrive/My Drive/tubes-of-dust-mricos/kauai-2sec/2403/"

path = "/content/gdrive/My Drive/tubes-of-dust-mricos/kauai-2sec/2403/"
for i in range(0,64):
    print("Writing file number {:02d}".format(i))
    librosa.output.write_wav(path+"{:02d}.wav".format(i), _G_z[i], 16000) 


In [0]:
!zip  "/content/gdrive/My Drive/tubes-of-dust-kauai-2sec-2403.zip" \
/content/gdrive/My\ Drive/tubes-of-dust-mricos/kauai-2sec/2403/*.wav

In [0]:
# Interpolate in latent space
import PIL
# CHANGE THESE to example IDs from the above cell
interp_a = 4
interp_b = 5

# CHANGE THIS to change number of intermediates
interp_n = 3

# Interpolate latent codes
_za, _zb = jewels[interp_a], jewels[interp_b]
_z_interp = []
for i in range(interp_n + 2):
  a = i / float(interp_n + 1)
  _z_interp.append((1-a) * _za + a * _zb)

# Concatenate for easier visualization
flat_pad = graph.get_tensor_by_name('flat_pad:0')
G_z_flat = graph.get_tensor_by_name('G_z_flat:0')[:, 0]
G_z_spec_padded = tf.pad(G_z_spec, [[0, 0], [0, 0], [0, 128]])
G_z_spec_padded = tf.transpose(G_z_spec_padded, [0, 2, 1])
G_z_spec_flat = tf.reshape(G_z_spec_padded, [-1, 256])
G_z_spec_flat = tf.transpose(G_z_spec_flat, [1, 0])[:, :-128]

# Generate
_G_z_flat, _G_z_spec = sess.run([G_z_flat, G_z_spec_flat], 
                                {z: _z_interp, flat_pad: 8192})


display(Audio(_G_z_flat, rate=16000))
file=path+"jewel4-to-jewel5.wav"
librosa.output.write_wav(file,_G_z_flat, 16000) 

In [0]:
!zip  "/content/gdrive/My Drive/tubes-of-dust-kauai-2sec-2403.zip" \
/content/gdrive/My\ Drive/tubes-of-dust-mricos/kauai-2sec/2403/jewel4-to-jewel5.wav